## Association Request

This notebook aims to give a brief explanation on  how the association request works.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import syft as sy
import torch as th

# start two different stacks
```
hagrid launch test_network_1 network to docker:9081
hagrid launch test_domain_1 domain to docker:9082
```

### Connect with Domain

In [ ]:
NETWORK_HOST_IP = "localhost:9081"

In [ ]:
domain = sy.login(email="info@openmined.org", password="changethis", port=9082)

### Request Node Association

In [ ]:
vpn_status = domain.vpn_status()

In [ ]:
vpn_status

In [ ]:
DOMAIN_VPN_IP = vpn_status["host"]["ip"]
for peer in vpn_status["peers"]:
    if peer["hostname"] == "test_network_1":
        NETWORK_VPN_IP = peer["ip"]

In [ ]:
print(DOMAIN_VPN_IP, NETWORK_VPN_IP)

In [ ]:
# we need to refactor this so that the VPN IPs are not required to be supplied
domain.apply_to_network(
    domain_vpn_ip=DOMAIN_VPN_IP,
    network_vpn_ip=NETWORK_VPN_IP
)

### Monitor Association Requests

In [ ]:
network = sy.login(email="info@openmined.org", password="changethis", port=9081)

In [ ]:
network.association.pandas()

### Accept/Deny Association Requests

In [ ]:
associations = network.association.all()

In [ ]:
request_id = int(associations[0]["association_id"])

In [ ]:
network.association[request_id].accept()

### Create Data and Tag

In [ ]:
import torch as th

In [ ]:
x = th.Tensor([1,2,3])

In [ ]:
x_ptr = x.send(domain, tags=["secret"])

In [ ]:
domain.store

### Use Domain's Proxy Client

In [ ]:
import syft as sy
import torch as th

In [ ]:
network = sy.login(email="info@openmined.org", password="changethis", port=9081)

In [ ]:
network.domains

In [ ]:
# you can use 3 different ways to get the proxy domain client

# with int row
domain_client = network.domains[0]

In [ ]:
# with id string
domain_client = network.domains["9a7fd94214e049f6871d3d21e274cd29"]

In [ ]:
# with UID object
from syft.core.common.uid import UID
id = UID("9a7fd94214e049f6871d3d21e274cd29")
domain_client = network.domains[id]

In [ ]:
# now we can actually use it

In [ ]:
domain_client

In [ ]:
domain_client.store

In [ ]:
secret_ptr = domain_client.store[0]

In [ ]:
try:
    res = secret_ptr.get(delete_obj=False)
    print("Now I can get", res)
except Exception as e:
    print("Guest cant get")

### Login to Remote Domain via Proxy Client

In [ ]:
domain_client.login(email="info@openmined.org", password="changethis")

In [ ]:
domain_client.logged_in

In [ ]:
try:
    res = secret_ptr.get(delete_obj=False)
    print("Now I can get", res)
except Exception as e:
    print("Guest cant get")

In [ ]:
domain_client.logout()

### Search the Network

In [ ]:
import syft as sy

In [ ]:
network = sy.login(email="info@openmined.org", password="changethis", port=9081)

In [ ]:
network.domains

In [ ]:
network.search(query=["secret"])